<a href="https://colab.research.google.com/github/ummessh/Legal-Document-Digitization/blob/main/Database/Database_for_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Database for OCR

In [ ]:
import sqlite3
from datetime import datetime
import json


In [ ]:
class OCRDocumentProcessor:
    def __init__(self, db_path="ocr_documents.db"):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.create_tables()

    def create_tables(self):
        # Main documents table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            file_name TEXT,
            file_path TEXT,
            processed_date DATETIME,
            status TEXT,
            total_pages INTEGER
        )''')

        # Text elements table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS text_elements (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            document_id INTEGER,
            page_number INTEGER,
            content TEXT,
            confidence_score FLOAT,
            x1 INTEGER,
            y1 INTEGER,
            x2 INTEGER,
            y2 INTEGER,
            FOREIGN KEY (document_id) REFERENCES documents(id)
        )''')

        # Table elements
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS table_elements (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            document_id INTEGER,
            page_number INTEGER,
            table_data TEXT,  -- JSON format for table data
            rows INTEGER,
            columns INTEGER,
            x1 INTEGER,
            y1 INTEGER,
            x2 INTEGER,
            y2 INTEGER,
            confidence_score FLOAT,
            FOREIGN KEY (document_id) REFERENCES documents(id)
        )''')

        # Stamps
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS stamps (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            document_id INTEGER,
            page_number INTEGER,
            stamp_type TEXT,  -- e.g., 'company', 'notary', 'government'
            x1 INTEGER,
            y1 INTEGER,
            x2 INTEGER,
            y2 INTEGER,
            confidence_score FLOAT,
            image_path TEXT,  -- path to extracted stamp image
            FOREIGN KEY (document_id) REFERENCES documents(id)
        )''')

        # Signatures
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS signatures (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            document_id INTEGER,
            page_number INTEGER,
            signature_type TEXT,  -- e.g., 'handwritten', 'digital'
            x1 INTEGER,
            y1 INTEGER,
            x2 INTEGER,
            y2 INTEGER,
            confidence_score FLOAT,
            image_path TEXT,  -- path to extracted signature image
            FOREIGN KEY (document_id) REFERENCES documents(id)
        )''')

        self.conn.commit()


In [ ]:

    def add_document(self, file_name, file_path, total_pages):
        self.cursor.execute('''
        INSERT INTO documents (file_name, file_path, processed_date, status, total_pages)
        VALUES (?, ?, ?, ?, ?)
        ''', (file_name, file_path, datetime.now(), 'processing', total_pages))

        doc_id = self.cursor.lastrowid
        self.conn.commit()
        return doc_id

    def add_text_element(self, doc_id, page_number, content, bbox, confidence):
        x1, y1, x2, y2 = bbox
        self.cursor.execute('''
        INSERT INTO text_elements
        (document_id, page_number, content, confidence_score, x1, y1, x2, y2)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (doc_id, page_number, content, confidence, x1, y1, x2, y2))
        self.conn.commit()

    def add_table_element(self, doc_id, page_number, table_data, rows, cols, bbox, confidence):
        x1, y1, x2, y2 = bbox
        table_json = json.dumps(table_data)
        self.cursor.execute('''
        INSERT INTO table_elements
        (document_id, page_number, table_data, rows, columns, x1, y1, x2, y2, confidence_score)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (doc_id, page_number, table_json, rows, cols, x1, y1, x2, y2, confidence))
        self.conn.commit()

    def add_stamp(self, doc_id, page_number, stamp_type, bbox, confidence, image_path):
        x1, y1, x2, y2 = bbox
        self.cursor.execute('''
        INSERT INTO stamps
        (document_id, page_number, stamp_type, x1, y1, x2, y2, confidence_score, image_path)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (doc_id, page_number, stamp_type, x1, y1, x2, y2, confidence, image_path))
        self.conn.commit()

    def add_signature(self, doc_id, page_number, sig_type, bbox, confidence, image_path):
        x1, y1, x2, y2 = bbox
        self.cursor.execute('''
        INSERT INTO signatures
        (document_id, page_number, signature_type, x1, y1, x2, y2, confidence_score, image_path)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (doc_id, page_number, sig_type, x1, y1, x2, y2, confidence, image_path))
        self.conn.commit()


In [ ]:
    # Query methods for retrieving elements
    def get_document_text(self, doc_id, page_number=None):
        if page_number:
            self.cursor.execute('''
            SELECT page_number, content, confidence_score
            FROM text_elements
            WHERE document_id = ? AND page_number = ?
            ORDER BY y1, x1
            ''', (doc_id, page_number))
        else:
            self.cursor.execute('''
            SELECT page_number, content, confidence_score
            FROM text_elements
            WHERE document_id = ?
            ORDER BY page_number, y1, x1
            ''', (doc_id,))
        return self.cursor.fetchall()

    def get_document_tables(self, doc_id, page_number=None):
        if page_number:
            self.cursor.execute('''
            SELECT page_number, table_data, rows, columns, confidence_score
            FROM table_elements
            WHERE document_id = ? AND page_number = ?
            ''', (doc_id, page_number))
        else:
            self.cursor.execute('''
            SELECT page_number, table_data, rows, columns, confidence_score
            FROM table_elements
            WHERE document_id = ?
            ORDER BY page_number
            ''', (doc_id,))
        return self.cursor.fetchall()

    def get_document_stamps(self, doc_id):
        self.cursor.execute('''
        SELECT page_number, stamp_type, confidence_score, image_path
        FROM stamps
        WHERE document_id = ?
        ORDER BY page_number
        ''', (doc_id,))
        return self.cursor.fetchall()

    def get_document_signatures(self, doc_id):
        self.cursor.execute('''
        SELECT page_number, signature_type, confidence_score, image_path
        FROM signatures
        WHERE document_id = ?
        ORDER BY page_number
        ''', (doc_id,))
        return self.cursor.fetchall()


In [ ]:
# Example usage
def process_ocr_results(ocr_results):
    db = OCRDocumentProcessor()

    # Add new document
    doc_id = db.add_document(
        file_name="contract.pdf",
        file_path="/path/to/contract.pdf",
        total_pages=5
    )

    # Process each page
    for page_num, page_data in enumerate(ocr_results, 1):
        # Add text elements
        for text_block in page_data['text']:
            db.add_text_element(
                doc_id=doc_id,
                page_number=page_num,
                content=text_block['text'],
                bbox=text_block['bbox'],
                confidence=text_block['confidence']
            )

        # Add tables
        for table in page_data['tables']:
            db.add_table_element(
                doc_id=doc_id,
                page_number=page_num,
                table_data=table['data'],
                rows=len(table['data']),
                cols=len(table['data'][0]),
                bbox=table['bbox'],
                confidence=table['confidence']
            )

        # Add stamps
        for stamp in page_data['stamps']:
            db.add_stamp(
                doc_id=doc_id,
                page_number=page_num,
                stamp_type=stamp['type'],
                bbox=stamp['bbox'],
                confidence=stamp['confidence'],
                image_path=stamp['image_path']
            )

        # Add signatures
        for signature in page_data['signatures']:
            db.add_signature(
                doc_id=doc_id,
                page_number=page_num,
                sig_type=signature['type'],
                bbox=signature['bbox'],
                confidence=signature['confidence'],
                image_path=signature['image_path']
            )

In [ ]:
    # Retrieve processed data
    text_content = db.get_document_text(doc_id)
    tables = db.get_document_tables(doc_id)
    stamps = db.get_document_stamps(doc_id)
    signatures = db.get_document_signatures(doc_id)

    return {
        'text': text_content,
        'tables': tables,
        'stamps': stamps,
        'signatures': signatures
    }